Metaclasse
- Cria classe de forma dinâmica
- Pode ser útil na criação de frameworks onde as classes precisam ter um comportamento específico.

In [61]:
class MeuMeta(type):
  def __new__(cls, nome, bases, dct):
    dct["novo_atributo"] = "Valor adicionado pela metaclasse"
    return super().__new__(cls, nome, bases, dct)

In [62]:
class MinhaClasse(metaclass=MeuMeta):
  pass

In [63]:
obj = MinhaClasse()
valor: str = getattr(obj, "novo_atributo")
# assert valor is not None
valor

'Valor adicionado pela metaclasse'

In [64]:
class ValidadorMeta(type):
  def __new__(cls, nome, bases, dict: dict):
    validacoes = dict.get("validacoes", {})
    for attr, tipo in validacoes.items():
      if not callable(tipo):
        raise TypeError(f"O tipo de validação para {attr} deve ser uma função")
      # Add função de validação
      def valida_func(self, value, attr=attr, tipo=tipo):
        if not isinstance(value, tipo):
          raise ValueError(f"{attr} deve ser do tipo {tipo.__name__}")
        self.__dict__[attr] = value
      # Renomeia a função para evitar problemas de escopo
      valida_func.__name__ = f"set_{attr}"
      dict[f"set_{attr}"] = valida_func
    return super().__new__(cls, nome, bases, dict)

In [65]:
class Usuario(metaclass=ValidadorMeta):
  validacoes = {
    "nome": str,
    "idade": int
  }
  
  def __init__(self, nome, idade):
    self.set_nome(nome)
    self.set_idade(idade)

In [66]:
try:
  user = Usuario("Caique", 30)
  print(f"Nome: {user.nome} Idade: {user.idade}")
  # Tentar adicionar valores inválidos
  user.set_idade(30) 
except ValueError as error:
  print(error)

Nome: Caique Idade: 30
